# <center>AI-Powered Energy Usage Optimization App</center>


# Data Set Information

- The datset contains 2075259 measurements gathered between December 2006 and November 2010 (47 months).
- Notes:
    1. (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3.

    2.  The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.

## Attribute Information:

1. date: Date in format dd/mm/yyyy

2. time: time in format hh:mm:ss

3. global_active_power: household global minute-averaged active power (in kilowatt)

4. global_reactive_power: household global minute-averaged reactive power (in kilowatt)

5. voltage: minute-averaged voltage (in volt)

6. global_intensity: household global minute-averaged current intensity (in ampere)

7. sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).

8. sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.

9. sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.


# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
# import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# 2. Load and Prepare Dataset

In [2]:
file_path = "dataset\household_power_consumption.txt"

In [13]:
parse_dates = lambda x, y: pd.to_datetime(x + ' ' + y, format='%d/%m/%Y %H:%M:%S')

df = pd.read_csv(
    file_path,
    sep=';',
    parse_dates=[["Date", "Time"]],
    date_parser=parse_dates,
    na_values='?',
    low_memory=False
)
df.head()

C:\Users\sawar\AppData\Local\Temp\ipykernel_2844\3412619025.py:3: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
C:\Users\sawar\AppData\Local\Temp\ipykernel_2844\3412619025.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


,Date_Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [14]:
# Keep only relevant sub-metering columns
df = df[["Date_Time", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]]
df.head()

,Date_Time,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,0.0,1.0,17.0
1,2006-12-16 17:25:00,0.0,1.0,16.0
2,2006-12-16 17:26:00,0.0,2.0,17.0
3,2006-12-16 17:27:00,0.0,1.0,17.0
4,2006-12-16 17:28:00,0.0,1.0,17.0


In [15]:
df = df.set_index("Date_Time").resample('D').sum().dropna()
df.head()

,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_Time,,,
2006-12-16,0.0,546.0,4926.0
2006-12-17,2033.0,4187.0,13341.0
2006-12-18,1063.0,2621.0,14018.0
2006-12-19,839.0,7602.0,6197.0
2006-12-20,0.0,2648.0,14063.0


# 3. Define Training Function

In [16]:
def train_and_save_model(series, name):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    sequence_length = 30
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X = np.array(X)
    y = np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=32))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    print(f"Training model for {name}...")
    model.fit(X, y, epochs=20, batch_size=16, verbose=0)

    model.save(f"model_{name}.h5")
    print(f"Saved model_{name}.h5")

# 4. Train Models for Each Appliance

In [ ]:
train_and_save_model(df["Sub_metering_1"], "kitchen")
train_and_save_model(df["Sub_metering_2"], "laundry")
train_and_save_model(df["Sub_metering_3"], "ac_heater")

print("All models trained and saved.")

d:\devanshu_college\capstone1\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training model for kitchen...


d:\devanshu_college\capstone1\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Saved model_kitchen.h5
Training model for laundry...


d:\devanshu_college\capstone1\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Saved model_laundry.h5
Training model for ac_heater...


Saved model_ac_heater.h5
✅ All models trained and saved.
